<a href="https://colab.research.google.com/github/CNG85-HW/CNG_Voodeo/blob/main/CNG_Voodeo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>